# Imports

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

---

# Carga de datos

In [22]:
features = pd.read_pickle("../../features/auctions_train_ventana_4.pkl")
targets = pd.read_pickle("../../targets/targets_ventana_4_auctions.pkl")
train = features.merge(targets, left_index=True, right_index=True)

---

# Preparación del set de datos para XGBoost

In [64]:
X, y = train.iloc[:,:-1], train.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [65]:
xg_reg = xgb.XGBRegressor(colsample_bytree = 0.7, learning_rate = 1,
                n_estimators = 100, n_jobs=6, max_depth=7, objective="reg:linear")

In [66]:
xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=6, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [67]:
preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 93545.206149


---

# Reborn

In [26]:
X, y = train.iloc[:,:-1], train.iloc[:,-1]
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)

param = {'max_depth':7,
         'eta':0.1,
         'silent':1,
         'objective':'reg:linear',
         'eval_metric' : 'rmse',
         'obj' : 'reg:squarederror',
         'colsample_bytree' : 1,
         'min_child_weight' : 0.5,
         'random_state' : 0,
         'reg_alpha' : 10,
         'reg_lambda' : 1,
         'scale_pos_weight' : 1,
         'learning_rate' : 0.01}


num_round = 5000
stopping = 20
bst = xgb.train(param, dtrain, num_round, evals=[(dtrain, "train"), (dtest, "test")], early_stopping_rounds=stopping)
# make prediction
#preds = bst.predict(dtest)

[0]	train-rmse:107923	test-rmse:109493
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 20 rounds.
[1]	train-rmse:107305	test-rmse:108895
[2]	train-rmse:106697	test-rmse:108307
[3]	train-rmse:106097	test-rmse:107729
[4]	train-rmse:105506	test-rmse:107158
[5]	train-rmse:104923	test-rmse:106596
[6]	train-rmse:104348	test-rmse:106044
[7]	train-rmse:103783	test-rmse:105497
[8]	train-rmse:103225	test-rmse:104956
[9]	train-rmse:102675	test-rmse:104425
[10]	train-rmse:102133	test-rmse:103899
[11]	train-rmse:101598	test-rmse:103383
[12]	train-rmse:101072	test-rmse:102875
[13]	train-rmse:100553	test-rmse:102372
[14]	train-rmse:100041	test-rmse:101882
[15]	train-rmse:99537	test-rmse:101395
[16]	train-rmse:99039.8	test-rmse:100918
[17]	train-rmse:98549.6	test-rmse:100446
[18]	train-rmse:98066.4	test-rmse:99982.1
[19]	train-rmse:97590.7	test-rmse:99527.4
[20]	train-rmse:97121.1	test-rmse:99076.4
[21]	train-rmse:9665